In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import json
import re
import os


In [2]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.loopnet.com/")

# Page load time. Can use time.sleep or WebDriverWait for dynamic content)
time.sleep(2)

In [3]:
# For sale or lease
search_for_sale = driver.find_element(By.CLASS_NAME, "search-types--for-sale")
search_for_sale.click()

# Property type
search_property_type = driver.find_element(By.NAME, "propertytypeforsale")
search_property_type.send_keys("Office")

# NOTE TO SELF: A better approach would be to navigate the dropdown menu and select the options

# Location
search_box = driver.find_element(By.NAME, "geography")
search_box.send_keys("Agoura Hills, CA")

search_box.send_keys(Keys.ENTER)
time.sleep(5)

In [43]:
# Extracting data (e.g., property titles)

# search_results = driver.find_elements(By.CLASS_NAME, "placards")

def extract_property_info(text):
    # Split the text into lines
    lines = text.splitlines()

    price = ""
    address = ""
    unit_square_foot = ""
    other_info = ""
    raw_square_foot = 0
    raw_price = 0



    for line in lines:
        # Check for price first
        if line[0] == "$":
            price = line
          
            # Remove the dollar sign and commas
            cleaned_text = line.replace('$', '').replace(',', '')
            # Convert the cleaned string to an integer
            raw_price = int(cleaned_text)

        # As long as the price is null, it will look for the address
        elif price == "":
            address += line + ", "

        # Check for unit/square foot
        elif "SF" in line:
            unit_square_foot = line

            # Regular expression pattern to match numbers with commas
            pattern = r'\d{1,3}(?:,\d{3})*'
            # Search for the pattern in the text
            match = re.search(pattern, text)

            if match:
                # Extract the number and remove any commas
                square_foot = int(match.group().replace(',', ''))
                raw_square_foot = square_foot
            else:
                print("No number found in the text.")
        else:
            other_info += line + ", "

    raw_price_per_square_foot = raw_price / raw_square_foot if raw_square_foot > 0 else 0
    formatted_price_per_square_foot = f"${raw_price_per_square_foot:.2f}" if raw_price_per_square_foot > 0 else "NA"

    return {
        "address": address,
        "price": price,
        "unit_square_foot": unit_square_foot,
        "other_info": other_info,
        "raw_price_per_square_foot": raw_price_per_square_foot,
        "formatted_price_per_square_foot": formatted_price_per_square_foot
    }



result = driver.find_element(By.CLASS_NAME, "placards")
ul_elements = result.find_elements(By.XPATH, './ul') 

properties = []
for ul in ul_elements:

    # li_elements = ul.find_elements(By.TAG_NAME, "li")
    li_elements = ul.find_elements(By.XPATH, './li') 

    for li in li_elements:
        property = {
          "listing_type": " For Sale",
          "address": "",
          "price": "",
          "unit_square_foot": "",
          "other_info": "",
          "raw_price_per_square_foot": 0,
          "formatted_price_per_square_foot": "NA"
        }

        if "AUCTION" in li.text:
            property["listing_type"] = "Auction"

        property_info = extract_property_info(li.text) 

        property["address"] = property_info["address"]
        property["price"] = property_info["price"]
        property["unit_square_foot"] = property_info["unit_square_foot"]
        property["other_info"] = property_info["other_info"]
        property["raw_price_per_square_foot"] = property_info["raw_price_per_square_foot"]
        property["formatted_price_per_square_foot"] = property_info["formatted_price_per_square_foot"]
        
        properties.append(property)


# Store data in a JSON file
# Ensure the data directory exists
os.makedirs('../data', exist_ok=True)

# Store data in a JSON file in the data folder
with open('../data/properties.json', 'w') as json_file:
    json.dump(properties, json_file, indent=4)


# Close the browser

# driver.quit()

In [46]:
# Convert the Json to a csv
df = pd.read_json('../data/properties.json')

# Sort the DataFrame by 'raw_price_per_square_foot', placing 0 values at the bottom
df['sort_key'] = df['raw_price_per_square_foot'].apply(lambda x: (x == 0, x))
df = df.sort_values(by='sort_key', ascending=True).drop(columns=['sort_key'])

df.to_csv('../data/properties.csv', index=False)




# # Convert the data into a Pandas DataFrame
# df = pd.DataFrame(titles, columns=["Property Title"])

# # Display the DataFrame
# df.head()

# # Optional: Save the data to a CSV file
# df.to_csv("loopnet_property_titles.csv", index=False)